In [22]:
import numpy as np
import math
import sys
from PIL import Image

In [23]:
# READ AND LOAD FROM FILE

def parse_camera_data(lines):
    camera_data = {}
    camera_data['position'] = [float(x) for x in lines[0].split()]
    camera_data['target'] = [float(x) for x in lines[1].split()]
    camera_data['normal'] = [float(x) for x in lines[2].split()]
    camera_data['aperture'] = float(lines[3])
    return camera_data


def parse_light_data(lines):
    num_lights = int(lines[0])
    light_data = []
    for i in range(1, num_lights + 1):
        light = {}
        light['position'] = [float(x) for x in lines[i].split()[:3]]
        light['color'] = [float(x) for x in lines[i].split()[3:6]]
        light['attenuation'] = [float(x) for x in lines[i].split()[6:]]
        light_data.append(light)
    return light_data

def parse_pigment_data(lines):
    num_pigments = int(lines[0])
    pigment_data = []
    line_index = 1
    for i in range(num_pigments):
        
        pigment = {}
        line = lines[line_index].split()
        
        pigment['type'] = line[0]
        
        if pigment['type'] == 'solid':
            pigment['color'] = [float(x) for x in line[1:4]]
            
        elif pigment['type'] == 'checker':
            pigment['color1'] = [float(x) for x in line[1:4]]
            pigment['color2'] = [float(x) for x in line[4:7]]
            pigment['cube_length'] = float(line[7])
            
        elif pigment['type'] == 'texmap':
            pigment['texture_file'] = line[1]
            pigment['P0'] = [float(x) for x in lines[line_index + 1].split()]
            pigment['P1'] = [float(x) for x in lines[line_index + 2].split()]
            line_index += 2
            
        pigment_data.append(pigment)
        
        line_index += 1
    return pigment_data, line_index


def parse_finish_data(lines):
    num_finishes = int(lines[0])
    finish_data = []
    for i in range(1, num_finishes + 1):
        finish = {}
        line = lines[i].split()
        finish['ka'] = float(line[0])
        finish['kd'] = float(line[1])
        finish['ks'] = float(line[2])
        finish['alpha'] = float(line[3])
        finish['kr'] = float(line[4])
        finish['kt'] = float(line[5])
        finish['ior'] = float(line[6])
        finish_data.append(finish)
    return finish_data


def parse_object_data(lines):
    num_objects = int(lines[0])
    object_data = []
    for i in range(1, num_objects + 1):
        obj = {}
        line = lines[i].split()
        obj['pigment_ref'] = int(line[0])
        obj['surface_ref'] = int(line[1])
        obj_type = line[2]
        if obj_type == 'sphere':
            obj['center'] = [float(x) for x in line[3:6]]
            obj['radius'] = float(line[6])
            obj['type'] = 'sphere'
        elif obj_type == 'polyhedron':
            num_faces = int(line[3])
            obj['faces'] = []
            for j in range(num_faces):
                face_coeffs = [float(x) for x in lines[i + 1 + j].split()]
                obj['faces'].append(face_coeffs)
            obj['type'] = 'polyhedron'
        object_data.append(obj)
    return object_data


def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    camera_data = parse_camera_data(lines[:4])
    light_data = parse_light_data(lines[4:])
    pigment_data, index = parse_pigment_data(lines[4 + len(light_data) + 1:])
    finish_data = parse_finish_data(lines[4 + len(light_data) + 1 + index:])
    object_data = parse_object_data(lines[4 + len(light_data) + 1 + index + len(finish_data) + 1:])

    return {
        'camera': camera_data,
        'lights': light_data,
        'pigments': pigment_data,
        'finishes': finish_data,
        'objects': object_data
    }

In [128]:
#CLASSES 

class Sphere:
    def __init__(self, center, radius, pigment_id, surface_id, color):
        self.center = np.array(center)
        self.radius = radius
        self.color = np.array(color)
        self.pigment_id = pigment_id
        self.surface_id = surface_id

    def ray_intersection(self, ray_origin, ray_direction):
        oc = ray_origin - self.center
        a = np.dot(ray_direction, ray_direction)
        b = 2 * np.dot(oc, ray_direction)
        c = np.dot(oc, oc) - self.radius ** 2

        discriminant = b ** 2 - 4 * a * c

        if discriminant < 0:
            return np.inf

        t1 = (-b + np.sqrt(discriminant)) / (2 * a)
        t2 = (-b - np.sqrt(discriminant)) / (2 * a)

        if t1 >= 0 and t2 >= 0:
            return min(t1, t2)
        elif t1 >= 0:
            return t1
        elif t2 >= 0:
            return t2
        else:
            return np.inf

    def get_pigment_color(self, hit_point, texture, checkerTexture):
        # solid
        if self.pigment_id == 2:
            return self.color
        
        # checker
        if self.pigment_id == 1:
            return checkerTexture.get_color(hit_point)

        # texmap
        u = np.arctan2(hit_point[2], hit_point[0]) / (2 * np.pi) + 0.5
        v = np.arccos(hit_point[1] / self.radius) / np.pi
        return texture.get_color(u,v)
#         texture = textures[self.pigment_id]
#         return texture.get_color(hit_point)


class Polyhedron:
    def __init__(self, faces, color, pigment_id, surface_id):
        self.faces = faces
        self.color = np.array(color)
        self.surface_id = surface_id
        self.pigment_id = pigment_id 

    def ray_intersection(self, ray_origin, ray_direction):
        t_min = np.inf

        for face in self.faces:
            plane_normal = face[:3]
            plane_distance = face[3]

            denominator = np.dot(plane_normal, ray_direction)

            if abs(denominator) > 1e-6:
                t = -np.dot(plane_normal, ray_origin - plane_distance) / denominator

                if t > 0 and t < t_min:
                    intersection_point = ray_origin + t * ray_direction

                    inside_halfspaces = True
                    for other_face in self.faces:
                        other_normal = other_face[:3]
                        other_distance = other_face[3]

                        if np.dot(other_normal, intersection_point) - other_distance < -1e-6:
                            inside_halfspaces = False
                            break

                    if inside_halfspaces:
                        t_min = t

        if t_min == np.inf:
            return np.inf
        else:
            return t_min
        
    def get_pigment_color(self, hit_point, texture, checkerTexture):
        # solid
        if self.pigment_id == 2:
            return self.color
        
         # checker
        if self.pigment_id == 1:
            return checkerTexture.get_color(hit_point)

        # texmap
        u = np.arctan2(hit_point[2], hit_point[0]) / (2 * np.pi) + 0.5
        v = np.arccos(hit_point[1] / self.radius) / np.pi
        return texture.get_color(u,v)
    
class Camera:
    def __init__(self, position, target, orientation, aperture):
        self.position = np.array(position)
        self.target = np.array(target)
        self.orientation = np.array(orientation)
        self.aperture = aperture

        self.forward = self.target - self.position
        self.forward = self.forward / np.linalg.norm(self.forward)

        right = np.cross(self.forward, self.orientation)
        self.right = right / np.linalg.norm(right)

        self.up = np.cross(self.right, self.forward)

    def generate_ray_direction(self, x, y, width, height):
        aspect_ratio = width / height
        fov_tan = np.tan(self.aperture / 2)
        normalized_x = (2 * (x + 0.5) / width - 1) * fov_tan * aspect_ratio
        normalized_y = (1 - 2 * (y + 0.5) / height) * fov_tan

        direction = self.forward + self.right * normalized_x + self.up * normalized_y
        return direction / np.linalg.norm(direction)


class Light:
    def __init__(self, position, color, attenuation):
        self.position = np.array(position)
        self.color = np.array(color)

        # coeficiente constante de atenuação, atenuação proporcional à distância da fonte da luz e coeficiente de 
        # e coeficiente de atenuação proporcional ao quadrado da distância da fonte da luz
        self.attenuation = np.array(attenuation) 

        
class Texture:
    def __init__(self, image_path, p0, p1):
        self.image = Image.open(image_path).convert("RGB")
        self.width = self.image.width
        self.height = self.image.height

    def get_color(self, u, v):
        x = int(u * self.width) % self.width
        y = int(v * self.height) % self.height
        return self.image.getpixel((x, y))
#     def __init__(self, image_path, p0, p1):
#         self.image = Image.open(image_path).convert("RGB")
#         self.width = self.image.width
#         self.height = self.image.height
#         self.p0 = np.array(p0)
#         self.p1 = np.array(p1)

#     def get_color(self, hit_point):
#         homogeneous_hit_point = np.append(hit_point, 1)  # Append 1 for homogeneous coordinates
#         s = np.dot(self.p0, homogeneous_hit_point)
#         r = np.dot(self.p1, homogeneous_hit_point)

#         x = int(s * self.width) % self.width
#         y = int(r * self.height) % self.height

#         return self.image.getpixel((x, y))
    
class CheckerTexture:
    def __init__(self, color1, color2, cube_length):
        self.color1 = np.array(color1)
        self.color2 = np.array(color2)
        self.cube_length = cube_length

    def get_color(self, point):
        scaled_x = int(point[0] / self.cube_length) % 2
        scaled_y = int(point[1] / self.cube_length) % 2
        scaled_z = int(point[2] / self.cube_length) % 2

        if (scaled_x + scaled_y + scaled_z) % 2 == 0:
            return self.color1
        else:
            return self.color2

In [116]:
#RAY TRACER
def trace_ray(ray_origin, ray_direction, objects, lights, texture, checkerTexture):
    closest_t = np.inf
    closest_object = None

    for obj in objects:
        t = obj.ray_intersection(ray_origin, ray_direction)
        if t < closest_t:
            closest_t = t
            closest_object = obj

    if closest_object is None:
        return np.array([0, 0, 0])

    hit_point = ray_origin + ray_direction * closest_t

    if isinstance(closest_object, Sphere):
        normal = (hit_point - closest_object.center) / closest_object.radius
    elif isinstance(closest_object, Polyhedron):
        normal = closest_object.faces[0][:3]  # Use the normal of the first face as an approximation

    # Ambient color
    ambient_color = [
        int(closest_object.get_pigment_color(hit_point, texture, checkerTexture)[0]*0.1),
        int(closest_object.get_pigment_color(hit_point, texture, checkerTexture)[1]*0.1),
        int(closest_object.get_pigment_color(hit_point, texture, checkerTexture)[2]*0.1)
    ]

    # Diffuse color
    diffuse_color = np.array([0, 0, 0])
    for light in lights:
        light_direction = light.position - hit_point
        light_distance = np.linalg.norm(light_direction)
        light_direction = light_direction / light_distance

        # Shadow check
        shadow_origin = hit_point + light_direction * 0.001
        shadow_t = np.inf
        for obj in objects:
            t = obj.ray_intersection(shadow_origin, light_direction)
            if 0.001 < t < light_distance:
                shadow_t = t
                break

        if shadow_t == np.inf:
            diffuse_intensity = np.maximum(np.dot(normal, light_direction), 0)
            
            diffuse_effect = closest_object.get_pigment_color(hit_point, texture, checkerTexture) * light.color
            diffuse_effect[0] *= diffuse_intensity
            diffuse_effect[1] *= diffuse_intensity
            diffuse_effect[2] *= diffuse_intensity
            array = [int(diffuse_effect[0]) , int(diffuse_effect[1]), int(diffuse_effect[2])  ]
            diffuse_color += array


    color = ambient_color + diffuse_color
    return np.minimum(color, 255)


In [111]:
# LOAD FROM FILE
data = parse_file('scene.txt')

# print(data['camera'])
#print(data['lights'])

print(data['pigments'])
# print(data['finishes'])
#print(data['objects'])



[{'type': 'texmap', 'texture_file': 'rainbow1.ppm', 'P0': [0.0, 0.001, 0.0, 0.12], 'P1': [0.0, 0.0, 0.0, 0.0]}, {'type': 'checker', 'color1': [0.08, 0.25, 0.2], 'color2': [0.93, 0.83, 0.82], 'cube_length': 40.0}, {'type': 'solid', 'color': [1.0, 0.0, 0.5]}]


In [114]:
def main():

    # Scene configuration
    width = 800
    height = 600

    # LOAD FROM FILE
    data = parse_file('scene.txt')

    # Save Pigments
    solid_color = []
    for pigment in data['pigments']:

        if pigment.get('type') == 'solid':
            solid_color = pigment['color']

        elif pigment.get('type') == 'texmap':
            texture = Texture(pigment['texture_file'], pigment['P0'], pigment['P1'])
            
        else:
            checkerTexture = CheckerTexture(pigment['color1'], pigment['color2'], pigment['cube_length'])
    
    solid_color = [int(solid_color[0]*255), int(solid_color[0]*255), int(solid_color[0]*255)]
    
    # Create objects
    objects = []
    for obj in data['objects']:
        
        if (obj['pigment_ref'] == 2): # solid
            color = solid_color
        else: 
            color = None
        
        # shpere
        if obj['type'] == 'sphere':    
            s = Sphere(center=obj['center'], radius=obj['radius'], color=color, pigment_id=obj['pigment_ref'], surface_id=obj['surface_ref'])
            objects.append(s)

        # polyhedron
        else:
            polyhedron = Polyhedron(faces=obj['faces'], color=color, pigment_id=obj['pigment_ref'], surface_id=obj['surface_ref'])
            objects.append(polyhedron)
    
    
    # Create lights
    lights = []
    for i in range(len(data['lights'])):
        color = [int(data['lights'][i]['color'][0]*255), int(data['lights'][i]['color'][1]*255), int(data['lights'][i]['color'][2]*255)]
        light = Light(position=data['lights'][i]['position'], color=color, attenuation=data['lights'][i]['attenuation'])
        lights.append(light)


    # Create camera
    camera = Camera( position=data['camera']['position'], target=data['camera']['target'], orientation=data['camera']['normal'], aperture=math.radians(data['camera']['aperture']))

    # Create image
    image = np.zeros((height, width, 3), dtype=np.uint8)

    for y in range(height):
        for x in range(width):
            ray_direction = camera.generate_ray_direction(x, y, width, height)

            color = trace_ray(camera.position, ray_direction, objects, lights, texture, checkerTexture)

            image[y, x] = color

    image = Image.fromarray(image)
    image.save("output.png")
    image.show()


In [130]:
main()

In [106]:
def create_rainbow_ppm(width, height):
    image = [['0 0 0' for _ in range(width)] for _ in range(height)]

    # Rainbow colors
    colors = [
        (255, 71, 47),    # Red
        (255, 183, 91),  # Orange
        (255, 255, 0),  # Yellow
        (20, 255, 20),    # Green
        (20, 20, 255),    # Blue
        (75, 0, 130),   # Indigo
        (238, 130, 238) # Violet
    ]

    # Assign colors to each pixel based on gradient
    for x in range(width):
        color_index = x * (len(colors) - 1) // width
        color_start, color_end = colors[color_index], colors[color_index + 1]

        for y in range(height):
            ratio = y / (height - 1)
            r = int(color_start[0] * (1 - ratio) + color_end[0] * ratio)
            g = int(color_start[1] * (1 - ratio) + color_end[1] * ratio)
            b = int(color_start[2] * (1 - ratio) + color_end[2] * ratio)
            image[y][x] = f'{r} {g} {b}'

    # Write the image data to a PPM file
    with open('rainbow1.ppm', 'w') as file:
        file.write(f'P3\n{width} {height}\n255\n')
        for row in image:
            file.write(' '.join(row) + '\n')

# Specify the dimensions of the image
width = 800
height = 200

# Create the rainbow PPM file
create_rainbow_ppm(width, height)
